<a href="https://colab.research.google.com/github/HKAgrwl/ImageProcessing/blob/master/VoiceCloner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U scipy
!git clone https://github.com/jnordberg/tortoise-tts.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 1481 (delta 397), reused 392 (delta 392), pack-reused 1039
Receiving objects: 100% (1481/1481), 53.55 MiB | 35.56 MiB/s, done.
Resolving deltas: 100% (615/615), done.


In [2]:
%cd tortoise-tts

/content/tortoise-tts


In [8]:
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with

In [9]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices


tts = TextToSpeech()

import os
from google.colab import files
import time
import random as rnd

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:01 Time:  0:00:01


Done.


In [10]:
def loader(speech_name):
    custom_voice_folder = f"tortoise/voices/{speech_name}"
    if not os.path.exists(custom_voice_folder):
        os.makedirs(custom_voice_folder)
    for i, file_data in enumerate(files.upload().values()):
        with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
            f.write(file_data)
    return speech_name

In [11]:
def VoiceClone(text, cvm):
    i = rnd.random()
    preset = 'standard'
    voice_samples, conditioning_latents = load_voice(cvm)
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    torchaudio.save(f'generated-{cvm}+{i}.wav', gen.squeeze(0).cpu(), 24000)
    return gen.squeeze(0)
   # returning the voice tensor

In [1]:
threshold = 17
# Maximum number of words in a sentence

In [5]:
def text_splitter(text):
    # Code to split large text into smaller ones.
    text_out = []
    for chunk in text.split('. '):
        if text_out and len(chunk) + len(text_out[-1]) < threshold:
            text_out[-1] += ' ' + chunk + '.'
        else:
            text_out.append(chunk + '.')
    return text_out

In [4]:
def generate_voice(text, cvm):
    if len(text) <= threshold:
        return VoiceClone(text, cvm)
    else:
        text_chunks = text_splitter(text)
        generated_voices = []
        for sentence in text_chunks:
            voice = VoiceClone(sentence, cvm)
            generated_voices.append(voice)
        combined_voice = torch.cat(generated_voices, dim=1)
        torchaudio.save(f'combined_voice_{cvm}.wav', combined_voice, 24000)
        return combined_voice

In [13]:
# Load the custom voice
custom_voice_name = 'CustomVoiceName'
load = loader(custom_voice_name)

Saving 1.wav to 1.wav


In [14]:
# User input for text
input_text = input("Enter the text: ")

Enter the text: This is a test case


In [15]:
# Generate voice for the text
generated_voice = generate_voice(input_text, load)
torchaudio.save('generated_voice.wav', generated_voice, 24000)
IPython.display.Audio('generated_voice.wav', rate=24000)

Generating autoregressive samples..


100%|██████████| 16/16 [00:20<00:00,  1.27s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 16/16 [00:16<00:00,  1.01s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 200/200 [00:11<00:00, 17.42it/s]


Next two blocks need not be run.

In [4]:
# Generate voice for a small text
small_text = "Hello world! This is a small text."
generated_voice_small = generate_voice(small_text, load)
torchaudio.save('generated_voice_small.wav', generated_voice_small, 24000)
IPython.display.Audio('generated_voice.wav', rate=24000)

NameError: ignored

In [ ]:
# Generate voice for a large text
large_text = "Kazakhstan, officially the Republic of Kazakhstan, is a transcontinental landlocked country located mainly in Central Asia and partly in Eastern Europe. It borders Russia to the north and west, China to the east, Kyrgyzstan to the southeast, Uzbekistan to the south, and Turkmenistan to the southwest, with a coastline along the Caspian Sea. Its capital is Astana, known as Nur-Sultan from 2019 to 2022. Almaty, Kazakhstan's largest city, was the country's capital until 1997. Kazakhstan is the world's ninth-largest country by land area and the world's largest landlocked country. It has a population of 19 million people and one of the lowest population densities in the world, at fewer than 6 people per square kilometre"
generated_voice_large = generate_voice(large_text, load)
torchaudio.save('generated_voice_large.wav', generated_voice_large, 24000)
IPython.display.Audio('generated_voice_large.wav')

In [5]:
!python3 tortoise/do_tts.py --text "I'm going to speak this" --voice random --preset fast

Traceback (most recent call last):
  File "/content/tortoise-tts/tortoise/do_tts.py", line 25, in <module>
    tts = TextToSpeech(models_dir=args.model_dir)
  File "/content/tortoise-tts/tortoise/api.py", line 182, in __init__
    self.aligner = Wav2VecAlignment()
  File "/usr/local/lib/python3.10/dist-packages/TorToiSe-2.3.0-py3.10.egg/tortoise/utils/wav2vec_alignment.py", line 51, in __init__
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 1978, in from_pretrained
    state_dict = load_state_dict(resolved_archive_file)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 392, in load_state_dict
    return torch.load(checkpoint_file, map_location="cpu")
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 809, in load
    return _load(opened_zipfile, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1172, in _load


In [22]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding